## Example for callbacks + Memory profiling of verbs and workflow

In this example we show how to use the workflow callbacks to be able profile the memory usage of every verb

In [1]:
# Enable nesting asyncio since jupyter has a asyncio already running
import nest_asyncio

nest_asyncio.apply()


import json
import os

from typing import Any, Dict, Optional, List
from datashaper.execution.execution_node import ExecutionNode
from datashaper.table_store import TableContainer

from typing import List

from datashaper.workflow import Workflow

### Create a Callback class
This class needs to follows the WorkflowCallbacks Protocol, we use tracemalloc to create memory snapshots and trace the memory

In [2]:
import pandas as pd
import tracemalloc
import time
from collections import defaultdict


class MemoryProfilingCallbacks:
    def __init__(self):
        self._snapshots = defaultdict(list)
        self._peak_memory = defaultdict(list)
        self._timing = defaultdict(list)
        self._peak_memory = defaultdict(list)
        self._peak_start_workflow = 0
        self._peak_start_verb = 0
        self._workflow_start = 0
        self._verb_start = 0

    def on_workflow_start(self) -> None:
        """Called when the workflow starts."""
        tracemalloc.start()
        _, self._peak_start_workflow = tracemalloc.get_traced_memory()
        self._snapshots["all"].append(tracemalloc.take_snapshot())
        self._workflow_start = time.time()

    def on_step_start(self, node: ExecutionNode, inputs: Dict[str, Any]) -> None:
        """Called when a step starts."""
        # reset peak so we can get the peak during the verb execution
        self._snapshots[node.verb.name].append(tracemalloc.take_snapshot())
        _, self._peak_start_verb = tracemalloc.get_traced_memory()
        self._verb_start = time.time()

    def on_step_end(
        self, node: ExecutionNode, result: Optional[TableContainer]
    ) -> None:
        """Called when a step ends."""
        total_time = time.time() - self._verb_start
        self._timing[node.verb.name].append(total_time)
        self._snapshots[node.verb.name].append(tracemalloc.take_snapshot())
        # Get peak recorded during verb execution
        _, peak = tracemalloc.get_traced_memory()
        self._peak_memory[node.verb.name].append(peak - self._peak_start_verb)

    def on_workflow_end(self) -> None:
        """Called when the workflow ends."""
        total_time = time.time() - self._workflow_start
        self._timing["all"].append(total_time)
        self._snapshots["all"].append(tracemalloc.take_snapshot())
        _, peak = tracemalloc.get_traced_memory()
        self._peak_memory["all"].append(peak - self._peak_start_workflow)
        tracemalloc.stop()

    def get_snapshot_stats(self, sort_by="max"):
        stats = {}
        for verb, snapshots in self._snapshots.items():
            verb_stats = []
            for first, second in zip(snapshots[::2], snapshots[1::2]):
                stat_diff = second.compare_to(first, "lineno")
                diff_size = sum(stat.size_diff for stat in stat_diff)
                verb_stats.append(MemoryProfilingCallbacks.__bytes_to_mb(diff_size))
            stats[verb] = {
                "mean": sum(verb_stats) / len(verb_stats),
                "max": max(verb_stats),
                "min": min(verb_stats),
                "samples": len(verb_stats),
            }
        return pd.DataFrame(stats).transpose().sort_values(sort_by, ascending=False)

    def get_peak_stats(self, sort_by="max"):
        stats = {}
        for verb, peak in self._peak_memory.items():
            stats[verb] = {
                "mean": MemoryProfilingCallbacks.__bytes_to_mb(sum(peak) / len(peak)),
                "max": MemoryProfilingCallbacks.__bytes_to_mb(max(peak)),
                "min": MemoryProfilingCallbacks.__bytes_to_mb(min(peak)),
                "samples": len(peak),
            }
        return pd.DataFrame(stats).transpose().sort_values(sort_by, ascending=False)

    def get_time_stats(self, sort_by="max"):
        stats = {}
        for verb, times in self._timing.items():
            stats[verb] = {
                "mean": sum(times) / len(times),
                "max": max(times),
                "min": min(times),
                "samples": len(times),
            }
        return pd.DataFrame(stats).transpose().sort_values(sort_by, ascending=False)

    def get_detailed_view(self):
        df_json = defaultdict(list)
        for verb, snapshot in self._snapshots.items():
            for sample, (first, second) in enumerate(
                zip(snapshot[::2], snapshot[1::2])
            ):
                stat_diff = second.compare_to(first, "lineno")
                for stat in stat_diff:
                    df_json["verb"].append(verb)
                    df_json["size_diff"].append(stat.size_diff)
                    df_json["size"].append(stat.size)
                    df_json["count_diff"].append(stat.count_diff)
                    df_json["count"].append(stat.count)
                    df_json["filename"].append(stat.traceback[0].filename)
                    df_json["lineno"].append(stat.traceback[0].lineno)
                    df_json["sample"].append(sample)
        return pd.DataFrame(df_json)

    @staticmethod
    def __bytes_to_mb(bytes):
        return bytes / 1024**2

In [3]:
FIXTURES_PATH = "../../../schema/fixtures/workflow"
TABLE_STORE_PATH = "../../../schema/fixtures/workflow_inputs"

memory_profiling_callbacks = MemoryProfilingCallbacks()


def get_verb_test_specs(root: str) -> List[str]:
    subfolders: List[str] = []
    for root, _, files in os.walk(root):
        if "workflow.json" in files:
            subfolders.append(root)
    return subfolders


def test_verbs_schema_input(fixture_path: str):
    with open(os.path.join(fixture_path, "workflow.json")) as schema:
        workflow = Workflow(
            schema=json.load(schema),
            input_path=TABLE_STORE_PATH,
        )

    workflow.run(workflow_callbacks=memory_profiling_callbacks)

In [4]:
for fixture_path in get_verb_test_specs(FIXTURES_PATH):
    test_verbs_schema_input(fixture_path)

/home/andresmor/Projects/datashaper/python/datashaper/datashaper/engine/verbs/convert.py:72: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  column_datetime = pd.to_datetime(column, errors="ignore")
/home/andresmor/Projects/datashaper/python/datashaper/datashaper/engine/verbs/convert.py:72: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  column_datetime = pd.to_datetime(column, errors="ignore")
/home/andresmor/Projects/datashaper/python/datashaper/datashaper/engine/verbs/convert.py:78: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  column_numeric = pd.to_numeric(column, errors="ignore")
/home/andresmor/Projects/datashaper/python/datashaper/data

Test print verb
    ID       Name  Employees     US
0    1  Microsoft     160000   True
1    2      Apple     150000   True
..  ..        ...        ...    ...
3    4     Amazon    1250000   True
4    5    Samsung     270000  False


In [5]:
memory_profiling_callbacks.get_snapshot_stats()

mean       max       min  samples
all              0.016333  0.207700  0.005172    148.0
fold             0.029311  0.117500  0.009782      6.0
convert          0.014538  0.094051  0.003598     15.0
unfold           0.031106  0.085790  0.009594      4.0
print            0.063319  0.063319  0.063319      1.0
union            0.034214  0.061944  0.006483      2.0
merge            0.010663  0.044469  0.006132     15.0
binarize         0.015506  0.042662  0.009157      7.0
difference       0.039237  0.039237  0.039237      1.0
join             0.021702  0.036840  0.014729      7.0
unroll           0.035782  0.035782  0.035782      1.0
onehot           0.019422  0.034151  0.013407      4.0
select           0.013519  0.033066  0.003618      3.0
fill             0.007246  0.032150  0.001953      6.0
unhot            0.022714  0.024974  0.020454      2.0
rollup           0.012957  0.024725  0.005470      3.0
spread           0.012904  0.023019  0.008410      4.0
bin              0.006824  0.019623  0.003822     17.0
window           0.019448  0.019448  0.019448      1.0
pivot            0.010971  0.017968  0.008568      6.0
intersect        0.016602  0.016602  0.016602      1.0
lookup           0.012994  0.012994  0.012994      1.0
filter           0.010296  0.012560  0.008730      7.0
orderby          0.005422  0.011877  0.004311      8.0
boolean          0.005779  0.011194  0.004511      6.0
impute           0.006002  0.010173  0.001831      2.0
aggregate        0.007760  0.009727  0.006813      4.0
derive           0.006037  0.009297  0.005055      8.0
dedupe           0.007022  0.007704  0.006340      2.0
drop             0.006093  0.007589  0.004597      2.0
strings.replace  0.007341  0.007341  0.007341      1.0
strings.upper    0.007195  0.007195  0.007195      1.0
unorder          0.004385  0.007133  0.003326      4.0
recode           0.006073  0.006073  0.006073      1.0
strings.lower    0.004768  0.004768  0.004768      1.0
groupby          0.004025  0.004318  0.003731      2.0
concat           0.004078  0.004078  0.004078      1.0
rename           0.003817  0.003817  0.003817      1.0
copy             0.003281  0.003281  0.003281      1.0
erase            0.002678  0.003120  0.002235      2.0
ungroup          0.000671  0.000671  0.000671      1.0

In [6]:
memory_profiling_callbacks.get_peak_stats()

mean       max       min  samples
all              0.023232  0.212424  0.005717    148.0
fold             0.041101  0.128269  0.020516      6.0
convert          0.015892  0.098181  0.004112     15.0
unfold           0.026965  0.079987  0.004741      4.0
union            0.045379  0.073210  0.017548      2.0
print            0.069079  0.069079  0.069079      1.0
join             0.044595  0.061277  0.035040      7.0
difference       0.054802  0.054802  0.054802      1.0
merge            0.017580  0.053172  0.010097     15.0
binarize         0.027253  0.051190  0.021309      7.0
unroll           0.043163  0.043163  0.043163      1.0
onehot           0.028473  0.042823  0.022891      4.0
intersect        0.035926  0.035926  0.035926      1.0
unhot            0.033809  0.033860  0.033758      2.0
select           0.015227  0.032447  0.006598      3.0
fill             0.007792  0.031952  0.002831      6.0
spread           0.016059  0.027440  0.008905      4.0
rollup           0.015457  0.024384  0.009308      3.0
filter           0.022000  0.024307  0.020551      7.0
lookup           0.024190  0.024190  0.024190      1.0
pivot            0.020373  0.024157  0.018611      6.0
window           0.020874  0.020874  0.020874      1.0
bin              0.007535  0.020299  0.004651     17.0
aggregate        0.014302  0.016989  0.013306      4.0
boolean          0.009999  0.014782  0.008501      6.0
orderby          0.007992  0.013920  0.006964      8.0
impute           0.008180  0.012541  0.003819      2.0
derive           0.007788  0.011832  0.005914      8.0
drop             0.008047  0.010084  0.006010      2.0
strings.replace  0.009717  0.009717  0.009717      1.0
strings.upper    0.008634  0.008634  0.008634      1.0
dedupe           0.007934  0.008622  0.007246      2.0
rename           0.008455  0.008455  0.008455      1.0
unorder          0.004902  0.008297  0.003635      4.0
recode           0.007493  0.007493  0.007493      1.0
strings.lower    0.006978  0.006978  0.006978      1.0
concat           0.006680  0.006680  0.006680      1.0
groupby          0.004116  0.004478  0.003754      2.0
erase            0.003944  0.004372  0.003517      2.0
copy             0.003856  0.003856  0.003856      1.0
ungroup          0.001245  0.001245  0.001245      1.0

In [7]:
memory_profiling_callbacks.get_time_stats()

mean       max       min  samples
all              0.013173  0.059589  0.000828    148.0
join             0.042037  0.054187  0.032328      7.0
binarize         0.032697  0.052202  0.021101      7.0
merge            0.015433  0.039304  0.007336     15.0
fold             0.019102  0.033014  0.012660      6.0
convert          0.005588  0.030703  0.001391     15.0
filter           0.023549  0.028906  0.018666      7.0
difference       0.027840  0.027840  0.027840      1.0
unhot            0.024410  0.024635  0.024185      2.0
onehot           0.016577  0.021322  0.014472      4.0
intersect        0.020867  0.020867  0.020867      1.0
unfold           0.018514  0.019749  0.017627      4.0
pivot            0.016604  0.018455  0.014350      6.0
union            0.012013  0.016987  0.007039      2.0
recode           0.013023  0.013023  0.013023      1.0
print            0.011796  0.011796  0.011796      1.0
spread           0.005627  0.010998  0.003082      4.0
rollup           0.006287  0.008937  0.003387      3.0
aggregate        0.007033  0.008225  0.004766      4.0
lookup           0.007205  0.007205  0.007205      1.0
unroll           0.006467  0.006467  0.006467      1.0
bin              0.004176  0.006371  0.002305     17.0
boolean          0.004657  0.005810  0.003550      6.0
strings.upper    0.004399  0.004399  0.004399      1.0
window           0.003838  0.003838  0.003838      1.0
select           0.002043  0.003536  0.001137      3.0
dedupe           0.002698  0.002889  0.002506      2.0
rename           0.002767  0.002767  0.002767      1.0
derive           0.002024  0.002762  0.001054      8.0
strings.replace  0.002749  0.002749  0.002749      1.0
drop             0.001981  0.002119  0.001844      2.0
fill             0.001136  0.002114  0.000625      6.0
orderby          0.001461  0.001826  0.000856      8.0
copy             0.001682  0.001682  0.001682      1.0
impute           0.001392  0.001624  0.001160      2.0
strings.lower    0.001444  0.001444  0.001444      1.0
unorder          0.000998  0.001240  0.000693      4.0
erase            0.000979  0.001158  0.000801      2.0
groupby          0.000784  0.001078  0.000490      2.0
concat           0.000944  0.000944  0.000944      1.0
ungroup          0.000032  0.000032  0.000032      1.0

In [8]:
dive_deep = memory_profiling_callbacks.get_detailed_view()

In [9]:
dive_deep[
    (dive_deep["verb"] == "unfold")
    & (dive_deep["sample"] == 0)
    & (dive_deep["filename"].str.contains("unfold"))
]

verb  size_diff  size  count_diff  count  \
15228  unfold       1104  1104           2      2   
15288  unfold        464   464           1      1   
15338  unfold        424   424           1      1   
15368  unfold        408   408           1      1   
15410  unfold        144   144           2      2   
15430  unfold         96    96           2      2   

                                                filename  lineno  sample  
15228  /home/andresmor/Projects/datashaper/python/dat...      28       0  
15288  /home/andresmor/Projects/datashaper/python/dat...      26       0  
15338  /home/andresmor/Projects/datashaper/python/dat...      21       0  
15368  /home/andresmor/Projects/datashaper/python/dat...      19       0  
15410  /home/andresmor/Projects/datashaper/python/dat...      22       0  
15430  /home/andresmor/Projects/datashaper/python/dat...      30       0